# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dikson,73.5069,80.5464,1.76,83,100,18.77,RU,2023-02-08 12:38:23
1,1,kapaa,22.0752,-159.3190,73.72,84,0,14.97,US,2023-02-08 12:38:24
2,2,umm lajj,25.0213,37.2685,69.04,42,100,19.42,SA,2023-02-08 12:38:24
3,3,mataura,-46.1927,168.8643,48.47,90,79,7.25,NZ,2023-02-08 12:38:25
4,4,coihaique,-45.5752,-72.0662,63.21,72,75,8.05,CL,2023-02-08 12:35:30


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

City_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)

# Display the map plot
City_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 93)& (city_data_df["Max Temp"] >= 75) & (city_data_df['Humidity'] <= 40)]

# Drop any rows with null values
ideal_data_df = city_data_df.dropna(how='any',axis=0)

# Display sample data
ideal_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dikson,73.5069,80.5464,1.76,83,100,18.77,RU,2023-02-08 12:38:23
1,1,kapaa,22.0752,-159.3190,73.72,84,0,14.97,US,2023-02-08 12:38:24
2,2,umm lajj,25.0213,37.2685,69.04,42,100,19.42,SA,2023-02-08 12:38:24
3,3,mataura,-46.1927,168.8643,48.47,90,79,7.25,NZ,2023-02-08 12:38:25
4,4,coihaique,-45.5752,-72.0662,63.21,72,75,8.05,CL,2023-02-08 12:35:30


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(weather_city)
hotel_df.pop("City_ID")
hotel_df.pop('Max Temp')
hotel_df.pop('Cloudiness')
hotel_df.pop('Wind Speed')
hotel_df.pop('Date')
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
0,port-cartier,50.0334,-66.8654,85,CA,
1,tuktoyaktuk,69.4541,-133.0374,75,CA,
2,atuona,-9.8000,-139.0333,74,PF,
3,avarua,-21.2078,-159.7750,74,CK,
4,klaksvik,62.2266,-6.5890,84,FO,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for index, row in cities_pd.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lon"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
           
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first airport from the results and store the details in the DataFrame
    try:
        cities_pd.loc[index, "Airport Name"] = response["features"][0]["properties"]["name"]
        cities_pd.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
        cities_pd.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        cities_pd.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
        cities_pd.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]
    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {cities_pd.loc[index, 'Airport Name']}")
        
    # Log the search results
    print(f"nearest airport from {city}: {cities_pd.loc[index, 'Airport Name']}")

# Display sample data
cities_pd

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":1
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row[1]
    lon = row[2]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    "Hotel Name",
    "Country"
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity"
)

# Display the map plot
hotel_plot